UMLS的数据分析

In [7]:
import jsonlines


instructions = list(jsonlines.open('data/instruction_dataall.jsonl'))
instructions = [ins for ins in instructions if ins['input'] and ins['output']]
print()

In [ ]:
import json 

js = json.load(open("data/ner_results_all.json"))
js[0]

In [ ]:
import jsonlines
import pandas as pd

instructions = list(jsonlines.open("../instruction_datasets/instruction_dataall.jsonl"))
df = pd.read_csv("./UMLS/umls_kg_filter.csv")
# mrdef = pd.read_csv("./UMLS/MRDEF_name.csv",error_bad_lines=False)


In [3]:
source_counts = df['source'].value_counts()
target_counts = df['target'].value_counts()
edge_counts = df['edge'].value_counts()
import plotly.express as px
px.pie(edge_counts, values=edge_counts.values, names=edge_counts.index, title='edge distribution')

In [5]:
from tqdm.auto import tqdm
import spacy
nlp = spacy.load("en_core_web_sm")
import random

i = random.choice(instructions)
prompt = i['input'].strip()+" "+i['output'].strip()
doc = nlp(prompt)
from spacy import displacy
displacy.render(doc, style="ent")

In [3]:
import pickle
from transformers import AutoTokenizer
input_ids_list, attention_mask_list, labels_list, hard_position_type_ids_list, prompts_list = pickle.load(open('data/EntityDataset_1000/ep_0.pkl','rb'))
tok = AutoTokenizer.from_pretrained('/home/cs/yangyuchen/yushengliao/Medical_LLM/FastChat/checkpoints/medical_llama_13b_chatv1.3/checkpoint-4974/')
dash_token="[DASH]"
tok.add_tokens([dash_token])
len(tok)

32001

In [53]:
import numpy as np
import torch
import pandas as pd


i = 0
input_ids=input_ids_list[i]
attention_mask=attention_mask_list[i]
labels=labels_list[i]
hard_position_type_ids=hard_position_type_ids_list[i]
prompt=prompts_list[i]


tokens = tok.batch_decode(input_ids)
tokens = [t.replace('\n', '\\n') for t in tokens]
tokens = [f"{t}-{int(hard_position_type_ids[i])}" for i,t in enumerate(tokens)]
attention_mask = attention_mask.numpy().tolist()
attention_mask = [[int(i) for i in row] for row in attention_mask]

matrix = attention_mask
row_index = tokens
col_index = tokens

max_row_length = max(len(row) for row in row_index)
max_col_length = max(len(col) for col in col_index)

# 打开文件
with open('vis_dataset.txt', 'w') as f:
    f.write(prompt+ '\n\n')
    f.write('attention_mask:\n0:non-entity tokens，1:entity tokens, 2:triplet tokens, 3:triplet target tokens\n\n')
    # 写入列索引
    f.write(" " * max_row_length + "  " + "  ".join(col.ljust(max_col_length) for col in col_index) + '\n')
    # 写入行索引和每行的值
    for i, row in enumerate(matrix):
        f.write(row_index[i].ljust(max_row_length) + "  " + "  ".join(str(val).ljust(max_col_length) for val in row) + '\n')
        
# 创建DataFrame对象
df = pd.DataFrame(matrix, index=row_index, columns=col_index)

# 将DataFrame写入CSV文件
df.to_csv('vis_dataset.csv')